# Exercise 6: Operator Splitting Methods

## Learning Objectives

By completing this exercise, you will:
- Understand different operator splitting strategies (Additive, Lie-Trotter, Strang)
- Analyze the accuracy and stability of splitting methods
- Compare splitting approaches with direct integration
- Investigate when operator splitting fails

---

## Introduction

Operator splitting allows solving complex PDEs by breaking them into simpler sub-problems. See the [wiki: Operator Splitting](https://github.com/AST-Course/AST5110/wiki/Operator-splitting) for background.

---

## Part 1: Operator Splitting Precision

### The Problem

Solve the advection equation with two velocity components:

$$\frac{\partial u}{\partial t} = -a \frac{\partial u}{\partial x} - b \frac{\partial u}{\partial x} \tag{1}$$

Following the setup from [Exercise 2b](https://github.com/AST-Course/AST5110/blob/main/notebooks/ex_2b.ipynb).

**Setup:**
- Domain: $x \in [-2.6, 2.6]$
- Parameters: $a = -0.7$, $b = -0.3$
- Periodic boundary conditions
- Initial condition:
$$u(x,t=t_0) = \cos^2\left(\frac{6\pi x}{5}\right) / \cosh(5x^2) \tag{2}$$
- Resolution: `nump = 256`, 100 time steps

### Task 1.1: Implement Four Approaches

Solve the evolution using:

| Method | Description | Implementation |
|--------|-------------|----------------|
| **1. Additive OS** | $u^{n+1} = u^n + \Delta t(A + B)u^n$ | `osp_LL_Add` |
| **2. Lie-Trotter OS** | $u^{n+1} = e^{B\Delta t}e^{A\Delta t}u^n$ | `osp_LL_Lie` |
| **3. Strang OS** | $u^{n+1} = e^{A\frac{\Delta t}{2}}e^{B\Delta t}e^{A\frac{\Delta t}{2}}u^n$ | `osp_LL_Strang` |
| **4. No splitting** | $\frac{\partial u}{\partial t} = -(a+b)\frac{\partial u}{\partial x}$ | Direct |

> **Suggestion:** Use the Lax method with `deriv_cent` for all cases. Ensure boundaries are properly handled.

### Task 1.2: Stability Analysis

Start with `cfl_cut = 0.4` and increase up to `0.9`.

**Questions:**
1. Which OS schemes are stable?
2. Which one is more diffusive? Why?

---

## Part 2: When Does Operator Splitting Fail?

### Task 2.1: Mixed Methods with Strang Splitting

Use Strang splitting from Part 1 but apply a **predictor-corrector** explicit method for one operator.

**Setup:**
- Resolution: `nump = 512`, 500 steps, `cfl_cut = 0.8`

The Hyman predictor-corrector method is already implemented in [`nm_lib`](https://github.com/AST-Course/nm_lib/tree/master/nm_lib/nm_ex/nm_lib_ex_6.py) (used in Bifrost).

**Implementation:** Fill in `osp_Lax_LH_Strang`.

**Question:** What do you notice?

### Task 2.2: Verify Splitting Consistency (Optional)

Apply the Hyman predictor-corrector method to the full Burgers equation and verify:

$$u^{n+1} = F\, u^n \Delta t \approx G\, u^n \Delta t + H\, u^n \Delta t$$

---

## Hints

<details>
<summary><b>Hint 1: Order of accuracy</b></summary>

- **Additive:** First-order accurate
- **Lie-Trotter:** First-order accurate
- **Strang:** Second-order accurate (symmetric splitting)
</details>

<details>
<summary><b>Hint 2: Why splitting can fail</b></summary>

When operators don't commute ($[A,B] \neq 0$), splitting introduces errors. Mixing methods with different numerical properties (e.g., Lax + predictor-corrector) can cause additional instabilities.
</details>

<details>
<summary><b>Hint 3: Strang splitting structure</b></summary>

```python
# Half step with operator A
u_half = step_A(u, dt/2)
# Full step with operator B  
u_full = step_B(u_half, dt)
# Half step with operator A
u_new = step_A(u_full, dt/2)
```
</details>